In [1]:
import scipy as sc
import numpy as np

In [2]:
A =[[0, 1, 1, 1],
    [1, 0, 0, 1],
    [1, 1, 0, 1],
    [1, 1, 1, 0]]

B = 0.67
t = 0
prob = 0
prod = 1
k = 0


while prob != 1:
    x = [sc.stats.norm.pdf(t, 0, 1), sc.stats.poisson.pmf(t, 1), sc.stats.bernoulli.pmf(t, 1, 1), sc.stats. poisson.pmf(t, 2)]
    xi = np.array(x)
    xi.transpose
    for i in range(len(A)):
        for j in range(len(A)-1):
            prod = prod*(1 - B*A[i][j]*xi[j])
    prob = 1 - prod
    

    if prob > 0.9999999:
            prob = 1
    if prob == 1:
        print(f'vértice foi convencido por t após {k} tempos')
        break
        
    else:
        print(f'vértice sucetivel a t com probabilidade atual de {prob} no tempo {k}')
           
    t = t + 0.0001
    k = k+1

vértice sucetivel a t com probabilidade atual de 0.8317017167391456 no tempo 0
vértice sucetivel a t com probabilidade atual de 0.9337976076800899 no tempo 1
vértice sucetivel a t com probabilidade atual de 0.9739583985296312 no tempo 2
vértice sucetivel a t com probabilidade atual de 0.989756185510352 no tempo 3
vértice sucetivel a t com probabilidade atual de 0.9959704575220857 no tempo 4
vértice sucetivel a t com probabilidade atual de 0.9984149250850479 no tempo 5
vértice sucetivel a t com probabilidade atual de 0.9993764893530691 no tempo 6
vértice sucetivel a t com probabilidade atual de 0.9997547336401428 no tempo 7
vértice sucetivel a t com probabilidade atual de 0.9999035211467338 no tempo 8
vértice sucetivel a t com probabilidade atual de 0.9999620487293942 no tempo 9
vértice sucetivel a t com probabilidade atual de 0.9999850713494037 no tempo 10
vértice sucetivel a t com probabilidade atual de 0.9999941276107314 no tempo 11
vértice sucetivel a t com probabilidade atual de 0.